## Aviation Accident Risk Analysis: Data-Driven Recommendations for Safer Investments
This project explores historical aviation accident data to identify patterns, contributing factors, and risk profiles associated with various aircraft models, flight conditions, and operational phases. By integrating accident records with regulatory data, weather conditions, and aircraft registration details, I aim to uncover actionable insights that support strategic decision-making—particularly for stakeholders assessing aircraft safety before investment or deployment.

Through a combination of statistical techniques and visual analytics, this analysis reveals key trends spanning decades of incidents. The ultimate goal: to deliver **at least three concrete, data-backed business recommendations** that enhance aviation safety and reduce investment risk for operators, insurers, and aviation decision-makers.

## Guiding Questions for Analysis

To shape meaningful business recommendations and uncover the underlying factors contributing to aviation accidents, the following key questions will guide me in my analysis:

1. **Which aircraft models are associated with the highest and lowest accident rates, and how do these rates compare when normalized by fleet size or registration volume?**  
   *→ Informs investment risk by identifying safer aircraft models.*

2. **What role do weather conditions play in aviation accidents, and which specific weather types are most frequently linked to severe outcomes?**  
   *→ Supports operational planning and risk mitigation under adverse weather.*

3. **Are there identifiable trends in accidents across different phases of flight (e.g., takeoff, cruise, landing), and do these vary by aircraft type or operator category?**  
   *→ Guides targeted safety interventions at high-risk phases.*

4. **To what extent do regulatory or maintenance-related issues contribute to accident frequency or severity?**  
   *→ Informs policy adjustments and helps rank compliance risk across aircraft categories.*

5. **Have accident patterns shifted over time, and what does this reveal about the effectiveness of safety regulations or technological advancements?**  
   *→ Tracks progress and identifies areas needing continued focus.*

6. **Are there regional or geographical patterns in accident occurrence, especially in relation to weather or regulation enforcement?**  
   *→ Offers strategic insight for operators expanding into new territories.*

## PHASE ONE:  Data Understanding

In this section, i will dive into a comprehensive examination of all datasets i will use in the project. The goal is to assess their structure, contents, and quality — and begin identifying how they can be integrated to support meaningful analysis and actionable insights.

---

###  Objectives

- Understand the schema, variables, and value distributions in each dataset.
- Assess data quality: missing values, inconsistencies, encoding issues.
- Identify relationships and join keys across datasets.
- Define preprocessing needs for each dataset.

---

###  Approach

#### 1. **Main Exploration (Aviation DAta)**
- Load the aviation accident dataset.
- Inspect variable types and value ranges.
- Identify missing or inconsistent values.
- Explore time, location, aircraft model, and severity distributions.

#### 2. **Explore Supplementary Data**
- Review each FAA data:
  - Are the values well-formatted?
  - Any obvious missing or invalid entries?
  - What columns are useful?

#### 3. **Plan for Dataset Integration**
- Identify common keys for joining:
  - `Registration.Number` ↔ `N-Number` (FAA)
  - `Model` ↔ `MODEL` (FAA)
  - Date + Lat/Lon proximity ↔ GHCND Weather
- Consider transformations (e.g., date parsing, coordinate matching).

---



In [1]:
#importing standard libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

## PART ONE: Core Data Set Understanding
The primary dataset for this project consists of detailed records of aviation accidents, capturing various attributes such as accident number, date, aircraft model, flight phase, location, injury severity, and more. This dataset serves as the backbone of my analysis and will help me uncover core patterns in accident frequency, severity, and causes.

Before diving into analysis, i will begin by examining the structure and content of this dataset to understand its variables, detect missing or inconsistent data, and identify potential areas for transformation. This step is critical in ensuring that my insights are grounded in clean, reliable, and meaningful data.

**Objectives:**
- Get familiar with the features (columns) present in the dataset  
- Check the completeness and data types of each feature  
- Identify key columns that will drive our analysis.
- Detect potential issues such as missing values, formatting inconsistencies, or ambiguous entries  

This understanding will guide the cleaning, enrichment, and merging steps to follow as i prepare this data for deeper analysis and cross-linking with the supplementary datasets.


In [2]:
#Loading the data
aviation_data = pd.read_csv("Data/Aviation-data/AviationData.csv", encoding='latin1')

In [3]:
#check the shape
aviation_data.shape

(88889, 31)

In [4]:
#preview of the first five rows
aviation_data.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [5]:
#check the last five rows
aviation_data.tail()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022
88885,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,...,Personal,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022
88887,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,...,Personal,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
88888,20221230106513,Accident,ERA23LA097,2022-12-29,"Athens, GA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,1.0,0.0,1.0,NaN,NaN,NaN,30-12-2022


In [6]:
#checking columns
aviation_data.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [7]:
#quick view of the data set
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [8]:
#Checking numerucal data
aviation_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Number.of.Engines,82805.0,1.146585,0.446510,0.0,1.0,1.0,1.0,8.0
Total.Fatal.Injuries,77488.0,0.647855,5.485960,0.0,0.0,0.0,0.0,349.0
Total.Serious.Injuries,76379.0,0.279881,1.544084,0.0,0.0,0.0,0.0,161.0
Total.Minor.Injuries,76956.0,0.357061,2.235625,0.0,0.0,0.0,0.0,380.0
Total.Uninjured,82977.0,5.325440,27.913634,0.0,0.0,1.0,2.0,699.0


In [9]:
#Checking categorical Data
aviation_data.describe(include='O').T

,count,unique,top,freq
Event.Id,88889,87951,20001212X19172,3
Investigation.Type,88889,2,Accident,85015
Accident.Number,88889,88863,CEN22LA149,2
Event.Date,88889,14782,1984-06-30,25
Location,88837,27758,"ANCHORAGE, AK",434
Country,88663,219,United States,82248
Latitude,34382,25592,332739N,19
Longitude,34373,27156,0112457W,24
Airport.Code,50132,10374,NONE,1488
Airport.Name,52704,24870,Private,240


In [10]:
#Checking for missing values
aviation_data.isna().any()

Event.Id                  False
Investigation.Type        False
Accident.Number           False
Event.Date                False
Location                   True
Country                    True
Latitude                   True
Longitude                  True
Airport.Code               True
Airport.Name               True
Injury.Severity            True
Aircraft.damage            True
Aircraft.Category          True
Registration.Number        True
Make                       True
Model                      True
Amateur.Built              True
Number.of.Engines          True
Engine.Type                True
FAR.Description            True
Schedule                   True
Purpose.of.flight          True
Air.carrier                True
Total.Fatal.Injuries       True
Total.Serious.Injuries     True
Total.Minor.Injuries       True
Total.Uninjured            True
Weather.Condition          True
Broad.phase.of.flight      True
Report.Status              True
Publication.Date           True
dtype: b



###  Findings
- The dataset successfully loaded using `latin1` encoding due to extended character sets in some fields.
- A preliminary inspection using `.head()` and `.tail()` confirms the structure is consistent across rows.

###  Columns & Features
- The dataset contains a wide range of features including:
  - Aircraft information (make, model, engine type, registration number, etc.)
  - Flight conditions (weather, phase of flight, purpose of flight)
  - Accident details (date, location, injury severity, aircraft damage, narrative)

- Column names are inconsistent and will require **standardization and renaming** for readability and usability in analysis.

###  Data Types and Initial Insights
- The `.info()` summary reveals a mixture of:
  - **Categorical features** such as `Injury.Severity`, `Weather.Condition`, and `Aircraft.Damage`
  - **Date fields** like `Event.Date`, which will be parsed into datetime format

### Missing Data
- A significant number of features contain **missing or null values**, particularly in:
  - latitude and longitude
  - Airport name and Code
  - Aircraft category
These issues will be addressed during the **Data Cleaning** phase.

---

This initial preview establishes a foundational understanding of the dataset. Further steps will involve cleaning, transforming, and preparing the data for analysis.


## PART TWO: Supplementary Dataset(s) Understanding

To enrich the core data set and support deeper, more actionable insights, i decided to research and found supplimentary data from Federal Aviation Administration (FAA) and U.S. State Codes to fortify my analysis. Each dataset serves a specific analytical purpose and will be preprocessed accordingly.

---

### FAA Aircraft Registration Data

- **Files:** `MASTER.txt`, `ENGINE.txt`, `ACFTREF.txt`, `DEALER.txt`, `DEREG.txt`, `DOCINDEX.txt`
- **Purpose:** Provides detailed metadata about aircraft including model specifications, engine details, and ownership history.

---

### FAA Regulations and Incident Data

- **Files:** Cleaned regulation dataset (CSV)
- **Purpose:** Captures regulatory environment and safety measures in place during various incidents.

---

### U.S. State Codes Dataset

- **File:** `US_States_Codes.csv`
- **Purpose:** Translates state abbreviations to full names and standard codes.

---

These supplementary datasets will be cleaned, normalized, and merged with the main accident data using common identifiers such as `Registration.Number`, `Model`, and `Event.Date`. This integration will unlock multi-dimensional insights and strengthen the final recommendations.


##  FAA Aircraft Registration Data Overview

To enrich the aviation accident dataset and gain deeper insight into aircraft-specific characteristics, we incorporate supplementary data provided by the FAA. These files contain detailed registration, technical, and deregistration records for civil aircraft in the United States. Below is a description of each dataset and its intended use in the project:

### 1. `MASTER.txt`
- **Description**: This file includes comprehensive records of all currently registered aircraft, with details such as registration numbers, manufacturer info, year of manufacture, type of registrant (e.g., individual, corporation), aircraft type, engine type, and airworthiness certification dates.
- **Usage**: We will use `MASTER.txt` to extract key aircraft metadata and merge it with the main accident dataset using the `N-NUMBER` (which corresponds to `Registration.Number`). This will allow analysis of accident trends based on aircraft age, type, ownership category, and certification status.

### 2. `ACFTREF.txt`
- **Description**: A reference file mapping manufacturer and model codes to their descriptive names, including weight class and engine type.
- **Usage**: This will be used to decode the `MFR MDL CODE` in the `MASTER.txt` file, enabling us to identify specific aircraft makes and models in a readable format. This is essential for evaluating accident patterns associated with certain aircraft types.

### 3. `ENGINE.txt`
- **Description**: Contains technical specifications of various aircraft engines, linked by engine model codes.
- **Usage**: We can link this to the engine code field in the `MASTER.txt` file (`ENG MFR MDL`) to analyze whether engine type or engine-specific characteristics correlate with accident severity or frequency.

### 4. `DEREG.txt`
- **Description**: Records of deregistered aircraft, including reasons and dates of deregistration.
- **Usage**: This file may help in identifying aircraft that were involved in an accident and subsequently deregistered. We can use this to validate the aircraft's operational status post-accident and examine patterns in deregistration reasons.

---

By leveraging these datasets, we can build a richer, aircraft-level profile for each accident, supporting more robust analysis and stronger business recommendations.


 ------

## FAA Aircraft Registration Data (MASTER.txt)

The MASTER.txt file provides comprehensive registration information for aircraft in the United States. It includes ownership details, aircraft identifiers, location of registrants, certification statuses, and model references that can be linked to technical aircraft data from ACFTREF.txt.



In [11]:
#loading the master file
master = pd.read_csv("supplimentary-data/ReleasableAircraft/MASTER.txt", delimiter=',', low_memory=False,)

In [12]:
master.shape

(303020, 35)

In [13]:
master.head()


,N-NUMBER,SERIAL NUMBER,MFR MDL CODE,ENG MFR MDL,YEAR MFR,TYPE REGISTRANT,NAME,STREET,STREET2,CITY,...,OTHER NAMES(2),OTHER NAMES(3),OTHER NAMES(4),OTHER NAMES(5),EXPIRATION DATE,UNIQUE ID,KIT MFR,KIT MODEL,MODE S CODE HEX,Unnamed: 34
0,100,5334,7100510,17003,1940,1,BENE MARY D ...,PO BOX 329,,KETCHUM,...,...,...,...,...,20270430,600060,,,A004B3,NaN
1,10000,10000,2130004,,,7,9AT LLC ...,511 WEDGEWOOD AVE,,NASHVILLE,...,...,...,...,...,20310831,1443200,,,A00725,NaN
2,10001,A28,9601202,67007,1928,1,STOOS ROBERT A ...,PO BOX 1056,,LAKELAND,...,...,...,...,...,20290228,432072,,,A00726,NaN
3,10004,T18208245,2072738,,,7,ETOS AIR LLC ...,PO BOX 288,,NEW LONDON,...,...,...,...,...,20290331,102879,,,A00729,NaN
4,10006,BG-72,1152020,17026,1955,1,COUTCHES ROBERT HERCULES DBA ...,550 AIRWAY BLVD,,LIVERMORE,...,...,...,...,...,20280229,480110,,,A0072B,NaN


In [14]:
master.tail()

,N-NUMBER,SERIAL NUMBER,MFR MDL CODE,ENG MFR MDL,YEAR MFR,TYPE REGISTRANT,NAME,STREET,STREET2,CITY,...,OTHER NAMES(2),OTHER NAMES(3),OTHER NAMES(4),OTHER NAMES(5),EXPIRATION DATE,UNIQUE ID,KIT MFR,KIT MODEL,MODE S CODE HEX,Unnamed: 34
303015,9ZR,2228,8680511,60020,,3,INTERNATIONAL AIR SERVICES INC TRUSTEE ...,123 W NYE LN STE 129,,CARSON CITY,...,...,...,...,...,20320531,183334,,,AC6CC9,NaN
303016,9ZS,20004,5760102,00000,1974,1,COLLINS BRIAN D ...,42 BOGART DR,,PETERSBURG,...,...,...,...,...,20280831,221481,,,AC6CCA,NaN
303017,9ZT,0088,2130001,17042,2001,7,VALIAIR LC TRUSTEE ...,OGDEN AIRPORT GATEWAY CENTER,4248 S 1650 W BLDG 1 STE 10,OGDEN,...,...,...,...,...,20290531,233847,,,AC6CCB,NaN
303018,9ZU,18-7028,7101828,41508,1959,1,FOWLER RONALD W ...,478 AIRPORT RD,,LOPEZ ISLAND,...,...,...,...,...,20271130,264074,,,AC6CCC,NaN
303019,9ZX,121,05655US,09050,1986,1,HOOVER MICHAEL B ...,N13699 320TH ST,,RIDGELAND,...,...,...,...,...,20300331,272135,,,AC6CCF,NaN


In [15]:
master.columns

Index(['N-NUMBER', 'SERIAL NUMBER', 'MFR MDL CODE', 'ENG MFR MDL', 'YEAR MFR',
       'TYPE REGISTRANT', 'NAME', 'STREET', 'STREET2', 'CITY', 'STATE',
       'ZIP CODE', 'REGION', 'COUNTY', 'COUNTRY', 'LAST ACTION DATE',
       'CERT ISSUE DATE', 'CERTIFICATION', 'TYPE AIRCRAFT', 'TYPE ENGINE',
       'STATUS CODE', 'MODE S CODE', 'FRACT OWNER', 'AIR WORTH DATE',
       'OTHER NAMES(1)', 'OTHER NAMES(2)', 'OTHER NAMES(3)', 'OTHER NAMES(4)',
       'OTHER NAMES(5)', 'EXPIRATION DATE', 'UNIQUE ID', 'KIT MFR',
       ' KIT MODEL', 'MODE S CODE HEX', 'Unnamed: 34'],
      dtype='object')

In [16]:
master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303020 entries, 0 to 303019
Data columns (total 35 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   N-NUMBER          303020 non-null  object 
 1   SERIAL NUMBER     303020 non-null  object 
 2   MFR MDL CODE      303020 non-null  object 
 3   ENG MFR MDL       303020 non-null  object 
 4   YEAR MFR          303020 non-null  object 
 5   TYPE REGISTRANT   303020 non-null  object 
 6   NAME              303020 non-null  object 
 7   STREET            303020 non-null  object 
 8   STREET2           303020 non-null  object 
 9   CITY              303020 non-null  object 
 10  STATE             303020 non-null  object 
 11  ZIP CODE          303020 non-null  object 
 12  REGION            303020 non-null  object 
 13  COUNTY            303020 non-null  object 
 14  COUNTRY           303020 non-null  object 
 15  LAST ACTION DATE  303020 non-null  int64  
 16  CERT ISSUE DATE   30

In [17]:
master.describe().T

,count,mean,std,min,25%,50%,75%,max
LAST ACTION DATE,303020.0,2.023313e+07,1.634905e+04,19711124.0,20230505.00,20230728.0,20231030.00,20250623.0
TYPE ENGINE,303020.0,2.318481e+00,2.559712e+00,0.0,1.00,1.0,2.00,11.0
MODE S CODE,303020.0,5.155580e+07,1.021610e+06,50000002.0,50603522.75,51467374.5,52432601.25,53373705.0
UNIQUE ID,303020.0,6.541841e+05,4.330911e+05,0.0,280599.75,601671.5,983170.25,1511187.0
Unnamed: 34,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
master.describe(include='O').T

,count,unique,top,freq
N-NUMBER,303020,303020,100,1
SERIAL NUMBER,303020,237442,001,1734
MFR MDL CODE,303020,43826,7102802,4230
ENG MFR MDL,303020,2464,,30828
YEAR MFR,303020,116,,60326
TYPE REGISTRANT,303020,9,1,127921
NAME,303020,196111,BANK OF UTAH TRUSTEE ...,1900
STREET,303020,170477,3511 SILVERSIDE RD STE 105,1995
STREET2,303020,3046,,292496
CITY,303020,14150,WILMINGTON,7869


In [19]:
master.isna().any()

N-NUMBER            False
SERIAL NUMBER       False
MFR MDL CODE        False
ENG MFR MDL         False
YEAR MFR            False
TYPE REGISTRANT     False
NAME                False
STREET              False
STREET2             False
CITY                False
STATE               False
ZIP CODE            False
REGION              False
COUNTY              False
COUNTRY             False
LAST ACTION DATE    False
CERT ISSUE DATE     False
CERTIFICATION       False
TYPE AIRCRAFT       False
TYPE ENGINE         False
STATUS CODE         False
MODE S CODE         False
FRACT OWNER         False
AIR WORTH DATE      False
OTHER NAMES(1)      False
OTHER NAMES(2)      False
OTHER NAMES(3)      False
OTHER NAMES(4)      False
OTHER NAMES(5)      False
EXPIRATION DATE     False
UNIQUE ID           False
KIT MFR             False
 KIT MODEL          False
MODE S CODE HEX     False
Unnamed: 34          True
dtype: bool


####  Findings:

- The data loaded successfully using `pd.read_csv()` with `delimiter=','`.
- `.head()` and `.tail()` checks confirm consistent formatting and no structural corruption across rows.
- All **35 columns** were correctly recognized and parsed.

####  Key Columns:

- **`N-NUMBER`**: FAA registration number; serves as a unique aircraft ID.
- **`MFR MDL CODE`**: Manufacturer/model code — links to `ACFTREF.txt` for aircraft technical data.
- **`ENG MFR MDL`**: Engine model/manufacturer — links to `ENGINE.txt` for engine specifications.
- **`YEAR MFR`**: Aircraft manufacturing year — useful for age profiling.
- **`TYPE REGISTRANT`, `NAME`, `STREET`, `CITY`, `STATE`, `ZIP CODE`**: Registrant details for identifying aircraft ownership and geographic distribution.
- **`CERTIFICATION`, `TYPE AIRCRAFT`, `TYPE ENGINE`, `STATUS CODE`**: Technical and regulatory attributes.
- **`AIR WORTH DATE`, `EXPIRATION DATE`**: Aircraft certification and registration validity.

#### Data Quality:

- No missing values were observed in the sample preview.
- One column, **`Unnamed: 34`**, appears to be empty and will be dropped during cleaning.
- Some column names (e.g., `' KIT MODEL'`) include leading/trailing whitespace and will be standardized.

---

####  Planned Usage:

This dataset will enhance the **main aviation accident dataset** by providing:

- Aircraft ownership and certification context.
- Insight into how factors like **aircraft age**, **registrant type**, or **certification status** relate to accident **frequency** or **severity**.
- Support for constructing **risk profiles** for different aircraft types based on their historical and regulatory data.



## FAA Aircraft Reference Data (ACFTREF.txt)

The ACFTREF.txt file contains structured reference data for aircraft, detailing the manufacturer, model, engine type, aircraft category, number of engines and seats, weight class, and certification information. This dataset is clean and consistent, with well-defined column names and no missing values, making it readily usable for merging and analysis.

In [20]:
#loading the acftref file
acftref = pd.read_csv("supplimentary-data/ReleasableAircraft/ACFTREF.txt", delimiter=',', low_memory=False)

In [21]:
acftref.shape

(92872, 14)

In [22]:
acftref.head()

,CODE,MFR,MODEL,TYPE-ACFT,TYPE-ENG,AC-CAT,BUILD-CERT-IND,NO-ENG,NO-SEATS,AC-WEIGHT,SPEED,TC-DATA-SHEET,TC-DATA-HOLDER,Unnamed: 13
0,0020901,AAR AIRLIFT GROUP INC,UH-60A,6,3,1,0,2,15,CLASS 3,0,,...,NaN
1,0030109,EXLINE ACE-C,ACE-C,4,1,1,1,1,1,CLASS 1,82,,...,NaN
2,003010D,DELEBAUGH,P,4,1,1,1,1,1,CLASS 1,82,,...,NaN
3,003010H,DAL PORTO,BABY ACE D,4,1,1,1,1,1,CLASS 1,82,,...,NaN
4,003010P,DUNN,BABY ACE,4,1,1,1,1,1,CLASS 1,82,,...,NaN


In [23]:
acftref.tail()

,CODE,MFR,MODEL,TYPE-ACFT,TYPE-ENG,AC-CAT,BUILD-CERT-IND,NO-ENG,NO-SEATS,AC-WEIGHT,SPEED,TC-DATA-SHEET,TC-DATA-HOLDER,Unnamed: 13
92867,9970230,ZLIN AVIATION S R O,SHOCK CUB,4,1,1,1,1,2,CLASS 1,0,,...,NaN
92868,9980000,ZLT ZEPPELIN LUFTSCHIFFTECHNIK,LZ N07-100,3,1,1,0,3,17,CLASS 1,0,,...,NaN
92869,9980002,ZLT ZEPPELIN LUFTSCHIFFTECHNIK,LZ NO7-101,3,1,1,0,3,15,CLASS 2,0,,...,NaN
92870,9999999,UNKNOWN,UNKNOWN,1,0,1,1,0,999,CLASS 4,0,,...,NaN
92871,99999XX,UNKNOWN,UNKNOWN,4,1,1,1,1,2,CLASS 1,112,,...,NaN


In [24]:
acftref.columns

Index(['CODE', 'MFR', 'MODEL', 'TYPE-ACFT', 'TYPE-ENG', 'AC-CAT',
       'BUILD-CERT-IND', 'NO-ENG', 'NO-SEATS', 'AC-WEIGHT', 'SPEED',
       'TC-DATA-SHEET', 'TC-DATA-HOLDER', 'Unnamed: 13'],
      dtype='object')

In [25]:
acftref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92872 entries, 0 to 92871
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CODE            92872 non-null  object 
 1   MFR             92872 non-null  object 
 2   MODEL           92872 non-null  object 
 3   TYPE-ACFT       92872 non-null  object 
 4   TYPE-ENG        92872 non-null  int64  
 5   AC-CAT          92872 non-null  int64  
 6   BUILD-CERT-IND  92872 non-null  int64  
 7   NO-ENG          92872 non-null  int64  
 8   NO-SEATS        92872 non-null  int64  
 9   AC-WEIGHT       92872 non-null  object 
 10  SPEED           92872 non-null  int64  
 11  TC-DATA-SHEET   92872 non-null  object 
 12  TC-DATA-HOLDER  92872 non-null  object 
 13  Unnamed: 13     0 non-null      float64
dtypes: float64(1), int64(6), object(7)
memory usage: 9.9+ MB


In [26]:
acftref.describe().T

,count,mean,std,min,25%,50%,75%,max
TYPE-ENG,92872.0,2.135100,2.533424,0.0,1.0,1.0,1.0,11.0
AC-CAT,92872.0,1.077849,0.383337,1.0,1.0,1.0,1.0,3.0
BUILD-CERT-IND,92872.0,0.960612,0.398659,0.0,1.0,1.0,1.0,2.0
NO-ENG,92872.0,1.096843,0.787298,0.0,1.0,1.0,1.0,80.0
NO-SEATS,92872.0,6.940617,38.077458,0.0,1.0,2.0,2.0,999.0
SPEED,92872.0,14.967008,43.823143,0.0,0.0,0.0,0.0,1125.0
Unnamed: 13,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
acftref.describe(include='O').T

,count,unique,top,freq
CODE,92872,92872,0020901,1
MFR,92872,67685,BOEING,1541
MODEL,92872,35654,CHALLENGER II,1026
TYPE-ACFT,92872,11,4,70847
AC-WEIGHT,92872,4,CLASS 1,87722
TC-DATA-SHEET,92872,227,,91728
TC-DATA-HOLDER,92872,145,...,91728


In [28]:
acftref.isna().any()

CODE              False
MFR               False
MODEL             False
TYPE-ACFT         False
TYPE-ENG          False
AC-CAT            False
BUILD-CERT-IND    False
NO-ENG            False
NO-SEATS          False
AC-WEIGHT         False
SPEED             False
TC-DATA-SHEET     False
TC-DATA-HOLDER    False
Unnamed: 13        True
dtype: bool

 ---

#### Findings:

- Successfully loaded using `pd.read_csv()` with `delimiter=','`.
- Data is clean with clearly labeled columns and no immediate signs of missing or malformed values.
- Column names are structured and self-descriptive, requiring minimal preprocessing.

####  Key Columns:

- **`CODE`**: Unique identifier for each aircraft model — can be linked to `MASTER.txt` via `MFR MDL CODE`.
- **`MFR`, `MODEL`**: Aircraft manufacturer and model — provides context for identifying specific aircraft configurations.
- **`TYPE-ACFT`, `TYPE-ENG`**: Encoded aircraft and engine types — useful for categorizing incidents by type.
- **`AC-CAT`**: Aircraft category (e.g., airplane, rotorcraft) — helpful for grouping and comparative analysis.
- **`NO-ENG`, `NO-SEATS`**: Details on aircraft engine count and seating capacity — key for estimating potential occupancy and accident impact.
- **`AC-WEIGHT`**: Aircraft weight classification (e.g., Class 1, Class 3) — used in understanding accident risk per weight class.

---

####  Planned Usage:

This dataset will serve as a **technical reference** for enriching the main aviation accident dataset. By linking through keys like `MFR MDL CODE`, it enables:

- Assessment of **aircraft-specific risk factors**, such as engine type or seating capacity.
- Enhanced ability to generate **data-backed safety insights** and recommendations based on aircraft configuration.



## FAA Engine Reference Data (ENGINE.txt)

The ENGINE.txt file contains reference data about aircraft engines registered with the FAA. It supplements the main dataset by providing technical specifications related to engine make, model, and performance attributes.

In [29]:
engine = pd.read_csv("supplimentary-data/ReleasableAircraft/ENGINE.txt", delimiter=',', low_memory=False)

In [30]:
engine.shape

(4719, 7)

In [31]:
engine.head()

,CODE,MFR,MODEL,TYPE,HORSEPOWER,THRUST,Unnamed: 6
0,0,NONE,NONE,0,0,0,NaN
1,401,A.C.E.,HIDR MARK III,1,95,0,NaN
2,402,A.C.E.,UPRI MARK III,1,100,0,NaN
3,450,AEROMOMENT,AM13 SERIES,8,100,0,NaN
4,452,AEROMOMENT,AM15 SERIES,8,117,0,NaN


In [32]:
engine.columns

Index(['CODE', 'MFR', 'MODEL', 'TYPE', 'HORSEPOWER', 'THRUST', 'Unnamed: 6'], dtype='object')

In [33]:
engine.tail()

,CODE,MFR,MODEL,TYPE,HORSEPOWER,THRUST,Unnamed: 6
4714,83358,KDE,7215XF,10,135,0,NaN
4715,83359,KDE,HACKER,10,495,0,NaN
4716,83360,KDE,10218XF-105,10,140,0,NaN
4717,99222,MAGICALL,MAGIDRIVE 75,10,350,0,NaN
4718,99999,AMA/EXPR,UNKNOWN ENG,,0,0,NaN


In [34]:
engine.describe().T

,count,mean,std,min,25%,50%,75%,max
CODE,4719.0,38758.422971,19237.250200,0.0,27016.5,41978.0,52549.5,99999.0
HORSEPOWER,4719.0,541.894469,939.991359,0.0,84.5,235.0,660.0,18250.0
THRUST,4719.0,4402.875185,13907.162193,0.0,0.0,0.0,0.0,115540.0
Unnamed: 6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
engine.describe(include='O').T

,count,unique,top,freq
MFR,4719,270,LYCOMING,865
MODEL,4719,4046,ALL MDLS A/B,8
TYPE,4719,12,1,2392


In [36]:
engine.isna().any()

CODE          False
MFR           False
MODEL         False
TYPE          False
HORSEPOWER    False
THRUST        False
Unnamed: 6     True
dtype: bool

###  Engine Reference File (`ENGINE.txt`)

---

####  Findings:

- File successfully read using `pd.read_csv()` with `delimiter='|'`.
- Columns are clean, consistently formatted, and intuitive.
- No missing values were identified in initial inspection.
- One extraneous column (`Unnamed: 6`) appears to be empty and will be dropped during preprocessing.

---

#### Columns Overview:

| Column Name   | Description |
|---------------|-------------|
| **`CODE`**         | Unique identifier for each engine model. Links to `ENG MFR MDL` in `MASTER.txt`. |
| **`MFR`**          | Engine manufacturer (e.g., Lycoming, Pratt & Whitney). |
| **`MODEL`**        | Engine model name/designation. |
| **`TYPE`**         | Numerical or coded value representing engine type. May require external decoding for interpretation. |
| **`HORSEPOWER`**   | Power output of the engine in horsepower. Useful for performance analysis. |
| **`THRUST`**       | Thrust power (likely in pounds-force) — relevant for jet and turbine engines. |
| **`Unnamed: 6`**   | Empty column (likely due to trailing delimiter in raw file); to be dropped. |

---

#### Usage Strategy:

This dataset will enhance the analysis by:

- **Profiling engine performance** (e.g., power-to-weight ratios, aircraft capability).
- Investigating **correlations between engine specs and accident frequency or severity**.
- Identifying **failure trends** across manufacturers and models for better safety recommendations.
- Supporting the development of **engine-specific risk metrics** for use in fleet management or policy planning.

After minor cleaning (dropping the empty column), this file is **analysis-ready**.


## FAA Deregistered Aircraft Data (`DEREG.txt`)

In [37]:
def handle_bad_line(bad_line):
    print("Bad line encountered:", bad_line)
    return None  # skip the bad line

dereg = pd.read_csv(
    "supplimentary-data/ReleasableAircraft/DEREG.txt",
    delimiter=',',
    engine='python',
    on_bad_lines=handle_bad_line
)




Bad line encountered: ['54096', '27-7405407                    ', '7102308', 'V ', 'RUSSELL INC                                       ', 'PO BOX SB 51503                  ', '                                 ', 'NASSAU N P        ', '  ', '0000      ', '41533', '1974', '1N        ', ' ', '   ', 'BS', '19740509', '20210112', '51555636', '8', 'BAHAMAS           ', '20240412', '20190729', '348 OYSTER RD                    ', '                                 ', 'NASSAU', ' NP        ', '  ', '0000      ', '   ', 'BS', '                                                  ', '                                                  ', '                                                  ', '                                                  ', '                                                  ', '                              ', '                    ', 'A6DB9E    ', '']


In [38]:
dereg.shape

(380593, 39)

In [39]:
dereg.head()

,N-NUMBER,SERIAL-NUMBER,MFR-MDL-CODE,STATUS-CODE,NAME,STREET-MAIL,STREET2-MAIL,CITY-MAIL,STATE-ABBREV-MAIL,ZIP-CODE-MAIL,...,COUNTRY-PHYSICAL,OTHER-NAMES(1),OTHER-NAMES(2),OTHER-NAMES(3),OTHER-NAMES(4),OTHER-NAMES(5),KIT MFR,KIT MODEL,MODE S CODE HEX,Unnamed: 38
0,1,121,1000102,V,KEMNITZER GEORGE E ...,29 FRANKLIN AVENUE,,NEWARK,OH,43055,...,,...,...,...,...,...,,,A00001,NaN
1,1,058B-5472,0191006,V,BRICKER JAMES C ...,,,,,,...,,...,...,...,...,...,,,A00001,NaN
2,10,6756,1150538,V,CIVIL AERONAUTICS ADMINISTRATION ...,DEPT OF COMMERCE,,WASHINGTON,DC,20310,...,,...,...,...,...,...,,,A0025A,NaN
3,10,155,5070132,V,AERONAUTICAL BRANCH ...,U S DEPARTMENT OF COMMERCE,,WASHINGTON,DC,20310,...,,...,...,...,...,...,,,A0025A,NaN
4,100,UNKNOWN,9600102,V,HALL MURRAY ...,3507 LEE AVE,,LITTLE ROCK,AR,72331,...,,...,...,...,...,...,,,A004B3,NaN


In [40]:
dereg.tail()

,N-NUMBER,SERIAL-NUMBER,MFR-MDL-CODE,STATUS-CODE,NAME,STREET-MAIL,STREET2-MAIL,CITY-MAIL,STATE-ABBREV-MAIL,ZIP-CODE-MAIL,...,COUNTRY-PHYSICAL,OTHER-NAMES(1),OTHER-NAMES(2),OTHER-NAMES(3),OTHER-NAMES(4),OTHER-NAMES(5),KIT MFR,KIT MODEL,MODE S CODE HEX,Unnamed: 38
380588,9ZN,4177,2130001,V,CIRRUS DESIGN CORP ...,4515 TAYLOR CIR,,DULUTH,MN,55811,...,,...,...,...,...,...,,,AC6CC6,NaN
380589,9ZS,47153,1182124,V,PREMIER ROTORS LLC ...,1013 CAPRITTO RD,,SAINT MARTINVILLE,LA,705826731,...,,...,...,...,...,...,,,AC6CCA,NaN
380590,9ZV,3793,1181511,29,L & R INVESTMENT PARTNERS LLC ...,160 GREENTREE DR STE 101,,DOVER,DE,199047620,...,,...,...,...,...,...,,,AC6CCD,NaN
380591,9ZW,0890100,05608IF,11,...,,,,,,...,,...,...,...,...,...,RANS DESIGNS INC,S-9 CHAOS,AC6CCE,NaN
380592,9ZZ,TJ-11,1152744,16,GARTKE GENE ...,24301 APPLE TREE LN,,PLAINFIELD,IL,605442260,...,,...,...,...,...,...,,,AC6CD1,NaN


In [41]:
dereg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380593 entries, 0 to 380592
Data columns (total 39 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   N-NUMBER               380593 non-null  object 
 1   SERIAL-NUMBER          380593 non-null  object 
 2   MFR-MDL-CODE           380593 non-null  object 
 3   STATUS-CODE            380593 non-null  object 
 4   NAME                   380593 non-null  object 
 5   STREET-MAIL            380593 non-null  object 
 6   STREET2-MAIL           380593 non-null  object 
 7   CITY-MAIL              380593 non-null  object 
 8   STATE-ABBREV-MAIL      380593 non-null  object 
 9   ZIP-CODE-MAIL          380593 non-null  object 
 10  ENG-MFR-MDL            380593 non-null  object 
 11  YEAR-MFR               380593 non-null  object 
 12  CERTIFICATION          380593 non-null  object 
 13  REGION                 380593 non-null  object 
 14  COUNTY-MAIL            380593 non-nu

In [42]:
dereg.describe(include='O').T

,count,unique,top,freq
N-NUMBER,380593,299436,2067V,9
SERIAL-NUMBER,380593,269835,1,3956
MFR-MDL-CODE,380593,51996,7100510,4633
STATUS-CODE,380593,41,V,185239
NAME,380593,190640,SALE REPORTED ...,33804
STREET-MAIL,380593,180215,,39907
STREET2-MAIL,380593,6099,,365698
CITY-MAIL,380593,21215,,38072
STATE-ABBREV-MAIL,380593,64,,43966
ZIP-CODE-MAIL,380593,87069,,40608


In [43]:
dereg.describe().T

,count,mean,std,min,25%,50%,75%,max
MODE-S-CODE,380593.0,5.156081e+07,1.010696e+06,0.0,50617725.0,51503467.0,52405530.0,53373707.0
Unnamed: 38,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
dereg.columns

Index(['N-NUMBER', 'SERIAL-NUMBER', 'MFR-MDL-CODE', 'STATUS-CODE', 'NAME',
       'STREET-MAIL', 'STREET2-MAIL', 'CITY-MAIL', 'STATE-ABBREV-MAIL',
       'ZIP-CODE-MAIL', 'ENG-MFR-MDL', 'YEAR-MFR', 'CERTIFICATION', 'REGION',
       'COUNTY-MAIL', 'COUNTRY-MAIL', 'AIR-WORTH-DATE', 'CANCEL-DATE',
       'MODE-S-CODE', 'INDICATOR-GROUP', 'EXP-COUNTRY', 'LAST-ACT-DATE',
       'CERT-ISSUE-DATE', 'STREET-PHYSICAL', 'STREET2-PHYSICAL',
       'CITY-PHYSICAL', 'STATE-ABBREV-PHYSICAL', 'ZIP-CODE-PHYSICAL',
       'COUNTY-PHYSICAL', 'COUNTRY-PHYSICAL', 'OTHER-NAMES(1)',
       'OTHER-NAMES(2)', 'OTHER-NAMES(3)', 'OTHER-NAMES(4)', 'OTHER-NAMES(5)',
       'KIT MFR', ' KIT MODEL', 'MODE S CODE HEX', 'Unnamed: 38'],
      dtype='object')

In [45]:
dereg.isna().any()

N-NUMBER                 False
SERIAL-NUMBER            False
MFR-MDL-CODE             False
STATUS-CODE              False
NAME                     False
STREET-MAIL              False
STREET2-MAIL             False
CITY-MAIL                False
STATE-ABBREV-MAIL        False
ZIP-CODE-MAIL            False
ENG-MFR-MDL              False
YEAR-MFR                 False
CERTIFICATION            False
REGION                   False
COUNTY-MAIL              False
COUNTRY-MAIL              True
AIR-WORTH-DATE           False
CANCEL-DATE              False
MODE-S-CODE              False
INDICATOR-GROUP          False
EXP-COUNTRY              False
LAST-ACT-DATE            False
CERT-ISSUE-DATE          False
STREET-PHYSICAL          False
STREET2-PHYSICAL         False
CITY-PHYSICAL            False
STATE-ABBREV-PHYSICAL    False
ZIP-CODE-PHYSICAL        False
COUNTY-PHYSICAL          False
COUNTRY-PHYSICAL          True
OTHER-NAMES(1)           False
OTHER-NAMES(2)           False
OTHER-NA

## Findings: FAA Deregistered Aircraft Data (`DEREG.txt`)

The `DEREG.txt` file presented initial loading challenges due to a malformed line in the dataset. To address this, a custom function was implemented to skip the corrupted row during file read-in using the Python engine. This allowed the dataset to load successfully without compromising the integrity of the rest of the data.

---

###  File Status

- **File successfully loaded** after handling a single bad line.
- **Delimiter:** `,`
- **Engine used:** `python` (to support custom bad-line handling)
- **Data Quality:** Relatively clean; most fields are well-structured and populated.
- **Next Steps:** Full inspection and cleaning will be performed during the data wrangling phase.

---

###  Columns Overview

The file contains detailed deregistration and historical aircraft information, including:

- `N-NUMBER`, `SERIAL-NUMBER`, `MFR-MDL-CODE`: Unique identifiers for aircraft tracking.
- `ENG-MFR-MDL`, `YEAR-MFR`, `CERTIFICATION`: Technical and regulatory aircraft details.
- `NAME`, `MAILING & PHYSICAL ADDRESSES`, `COUNTRY`, `STATE`: Owner/registrant contact information.
- `STATUS-CODE`, `CANCEL-DATE`, `AIR-WORTH-DATE`: Registration and airworthiness history.
- `MODE S CODE` & `HEX`: Avionics transponder identifiers.
- `OTHER-NAMES`: Additional ownership or alias records.
- `Unnamed: 38`: Appears to be empty and will likely be dropped during cleaning.

---

###  Usage in Analysis

This dataset will supplement the main aviation accident dataset by:

- Providing insights into **aircraft deregistration patterns**, potentially flagging risks for previously deregistered or non-airworthy aircraft.
- Supporting analysis of how **registration timelines and cancellation dates** correlate with accident occurrence.
- Enabling enhanced **ownership and certification history tracking**, useful for investigating compliance or systemic issues.


## FAA REGULATION DATA
This dataset contains information about changes to Federal Aviation Administration (FAA) regulations. It is structured and consistent, with minimal preprocessing required.

In [46]:
regulation = pd.read_csv('supplimentary-data/Regulation-data/all_current_ACs_as_of_2025-06-24.csv')

In [47]:
regulation.shape

(779, 5)

In [48]:
regulation.head()

,CHANGENUMBER,DATE,DOCUMENTNUMBER,OFFICE,TITLE
0,0.0,1984-01-30T06:00:00Z,103-7,AFS-820,The Ultralight Vehicle
1,0.0,2005-04-25T06:00:00Z,25.783-1A,ANM-115,Fuselage Doors and Hatches
2,0.0,1993-02-02T06:00:00Z,25.1523-1,ANM-111,Minimum Flightcrew
3,0.0,1996-09-10T06:00:00Z,25-20,ANM-111,"Pressurization, Ventilation and Oxygen Systems..."
4,0.0,2001-05-25T06:00:00Z,25.723-1,ANM-110,Shock Absorption Tests


In [49]:
regulation.tail()

,CHANGENUMBER,DATE,DOCUMENTNUMBER,OFFICE,TITLE
774,0.0,2010-09-29T06:00:00Z,150/5210-13C,AAS-300,Airport Water Rescue Plans and Equipment
775,NaN,2010-09-29T06:00:00Z,21-47,AIR-100,"Submittal Of Data To An ACO, A DER Or An ODA F..."
776,NaN,2010-09-29T06:00:00Z,21-45,AIR-100,Commercial Parts
777,NaN,2010-12-09T06:00:00Z,33.91-1,ANE-111,Engine System and Component Tests
778,NaN,2011-01-10T06:00:00Z,120-107,AFS-200,Use of Remote On-Ground Ice Detection System


In [50]:
regulation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779 entries, 0 to 778
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CHANGENUMBER    427 non-null    float64
 1   DATE            779 non-null    object 
 2   DOCUMENTNUMBER  779 non-null    object 
 3   OFFICE          779 non-null    object 
 4   TITLE           779 non-null    object 
dtypes: float64(1), object(4)
memory usage: 30.6+ KB


In [51]:
regulation.describe().T

,count,mean,std,min,25%,50%,75%,max
CHANGENUMBER,427.0,0.255269,0.818185,0.0,0.0,0.0,0.0,9.0


In [52]:
regulation.describe(include='O').T

,count,unique,top,freq
DATE,779,672,2008-10-24T06:00:00Z,8
DOCUMENTNUMBER,779,779,103-7,1
OFFICE,779,103,AAS-100,75
TITLE,779,779,The Ultralight Vehicle,1


In [53]:
regulation.columns

Index(['CHANGENUMBER', 'DATE', 'DOCUMENTNUMBER', 'OFFICE', 'TITLE'], dtype='object')

In [54]:
regulation.isna().any()

CHANGENUMBER       True
DATE              False
DOCUMENTNUMBER    False
OFFICE            False
TITLE             False
dtype: bool

## Findings 
using the `.head` and `.tail`, the data is consistent all through 

| Column Name     | Description                                                                 |
|------------------|-----------------------------------------------------------------------------|
| `CHANGENUMBER`   | Unique identifier for the regulation change. Missing in some rows.          |
| `DATE`           | Date the regulation change was recorded or issued.                          |
| `DOCUMENTNUMBER` | Official document number for the regulatory record.                         |
| `OFFICE`         | FAA office responsible for the change or publication.                       |
| `TITLE`          | Title or summary of the regulatory change or document.                      |

---

### Data Quality

- The file is **relatively clean and analysis-ready**.
- The only notable missing values are in the `CHANGENUMBER` column.
- **No corrupted or malformed lines** observed.
- Column naming is already **consistent and descriptive**.

---

### Usage in Analysis

This dataset will be used to:

- **Overlay regulatory changes over time** with accident trends, helping identify correlations between new rules and safety outcomes.
- **Associate specific regulation documents** with incident dates or aircraft models where applicable.
- **Enrich the narrative** around FAA oversight, identifying whether accidents occurred before or after relevant safety regulations were enacted.


## PHASE TWO: DATA CLEANING & WRANGLING 

In this section, I begin the data cleaning and wrangling phase of my analysis. After getting an overview of all datasets, I will now dig deeper into the structure and contents of the aviation accident data — the main dataset powering my analysis.

My goal here is to:
- Understand the **meaning and relevance** of each column
- Decide which features are **critical for analysis**, and which can be **dropped or transformed**
- Handle **missing values** using clear logic
- Create a **clean, well-structured dataset** ready for Exploratory Data Analysis (EDA) and risk modeling

---

###  Why Focus on U.S. Data?

The aviation accident dataset spans both domestic and international incidents from **1962 to 2023**. After analyzing the `Country` column, I found:

- Total records in dataset: **88,889**
- Records with `Country == "United States"`: **82,248**
- Proportion of U.S. data: 92.5%


Given that **over 92% of the data is U.S.-based**, it is statistically sound to anchor my cleaning and initial analysis on this subset. This choice ensures:
- High-quality and consistent data (due to FAA reporting standards)
- Easier cross-referencing with other FAA and registration datasets
- A more stable foundation for accurate risk modeling and business recommendations

---

###  What About the Non-U.S. (Diaspora) Data?

While my focus will be on U.S.-based data for the purposes of cleaning, modeling, and initial business recommendations, I will **not discard the international data**.

Instead, I will:
- Preserve a cleaned version of non-U.S. (diaspora) data separately
- Consider adding it in later as a **secondary insight layer**
- Allow for potential **interactive filtering in dashboards** (e.g., U.S. vs. Global view)

This approach ensures that my analysis is both **deep (U.S. focus)** and **scalable (global relevance)**.

---

### Next Steps in Cleaning

I will now:
1. Filter and work with U.S. records only (`Country == "United States"`)
2. Examine each column in detail
3. Handle missing values logically
4. Clean inconsistencies (e.g., in aircraft model names, date formats, injury reports)
5. Save a clean version of the dataset for further analysis

Once complete, this cleaned dataset will form the foundation for:
- Exploratory Data Analysis
- Aircraft risk profiling
- Visualizations and business intelligence recommendations



In [55]:
#make a copy of the aviation data set since its good practice not working on the actual data 
aviation_data_copy = aviation_data.copy()


## Column Name Meanings

| Column Name              | Meaning                                                                       |
| ------------------------ | ----------------------------------------------------------------------------- |
| `Investigation.Type`     | Whether the event was an "Accident" or "Incident". Accidents are more severe. |
| `Accident.Number`        | Unique ID for each event. Serves as the primary key.                          |
| `Event.Date`             | Date the accident or incident occurred.                                       |
| `Location`               | General description of where the event happened (e.g., city, area).           |
| `Country`                | Country where the event occurred.                                             |
| `Latitude`               | Geographic coordinate (north-south) of the event.                             |
| `Longitude`              | Geographic coordinate (east-west) of the event.                               |
| `Airport.Code`           | FAA/IATA code of the airport involved (if any).                               |
| `Airport.Name`           | Full name of the airport involved (if any).                                   |
| `Injury.Severity`        | Summary of the severity of injuries (e.g., Fatal, Serious, Minor).            |
| `Aircraft.damage`        | Description of damage sustained by the aircraft.                              |
| `Aircraft.Category`      | General category of aircraft (e.g., airplane, rotorcraft).                    |
| `Registration.Number`    | Aircraft registration number (like a license plate).                          |
| `Make`                   | Manufacturer of the aircraft (e.g., Boeing, Cessna).                          |
| `Model`                  | Specific model of the aircraft.                                               |
| `Amateur.Built`          | Indicates if the aircraft was amateur-built ("Yes" or "No").                  |
| `Number.of.Engines`      | Number of engines the aircraft had.                                           |
| `Engine.Type`            | Description of the aircraft’s engine type.                                    |
| `FAR.Description`        | FAA regulatory category under which the aircraft was operating.               |
| `Schedule`               | Indicates if the flight was scheduled or unscheduled.                         |
| `Purpose.of.flight`      | Reason or purpose for the flight (e.g., personal, training).                  |
| `Air.carrier`            | Name of the air carrier, if applicable (commercial flights).                  |
| `Total.Fatal.Injuries`   | Total number of people who died in the event.                                 |
| `Total.Serious.Injuries` | Total number of people with serious injuries.                                 |
| `Total.Minor.Injuries`   | Total number of people with minor injuries.                                   |
| `Total.Uninjured`        | Total number of people who were not injured.                                  |
| `Weather.Condition`      | Weather during the event (e.g., VMC, IMC, UNK).                               |
| `Broad.phase.of.flight`  | Phase of flight during which the event occurred (e.g., landing, taxi).        |
| `Report.Status`          | Indicates if the report is preliminary or final.                              |
| `Publication.Date`       | Date the report was published.                                                |

----
The meanings provide description of what each column entails, thus expanding my domain knowledge on the data set

In [56]:
#cleaning and renaming the columns 
aviation_data_copy.rename(columns={
    'Investigation.Type': 'Investigation_Type',
    'Accident.Number': 'Accident_Number',
    'Event.Date': 'Event_Date',
    'Airport.Code': 'Airport_Code',
    'Airport.Name': 'Airport_Name',
    'Injury.Severity': 'Injury_Severity',
    'Aircraft.damage': 'Aircraft_Damage',
    'Aircraft.Category': 'Aircraft_Category',
    'Registration.Number': 'Registration_Number',
    'Make': 'Aircraft_Make',
    'Model': 'Aircraft_Model',
    'Amateur.Built': 'Amateur_Built',
    'Number.of.Engines': 'Number_of_Engines',
    'Engine.Type': 'Engine_Type',
    'FAR.Description': 'FAR_Description',
    'Schedule': 'Schedule_Type',
    'Purpose.of.flight': 'Purpose_of_Flight',
    'Air.carrier': 'Air_Carrier',
    'Total.Fatal.Injuries': 'Fatal_Injuries',
    'Total.Serious.Injuries': 'Serious_Injuries',
    'Total.Minor.Injuries': 'Minor_Injuries',
    'Total.Uninjured': 'Uninjured',
    'Weather.Condition': 'Weather_Condition',
    'Broad.phase.of.flight': 'Phase_of_Flight',
    'Report.Status': 'Report_Status',
    'Publication.Date': 'Publication_Date'
}, inplace=True)

aviation_data_copy.columns

Index(['Event.Id', 'Investigation_Type', 'Accident_Number', 'Event_Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport_Code',
       'Airport_Name', 'Injury_Severity', 'Aircraft_Damage',
       'Aircraft_Category', 'Registration_Number', 'Aircraft_Make',
       'Aircraft_Model', 'Amateur_Built', 'Number_of_Engines', 'Engine_Type',
       'FAR_Description', 'Schedule_Type', 'Purpose_of_Flight', 'Air_Carrier',
       'Fatal_Injuries', 'Serious_Injuries', 'Minor_Injuries', 'Uninjured',
       'Weather_Condition', 'Phase_of_Flight', 'Report_Status',
       'Publication_Date'],
      dtype='object')

In [57]:
def clean_column_names(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace('.', '_', regex=False)
        .str.replace(' ', '_', regex=False)
    )
    return df

aviation_data_copy = clean_column_names(aviation_data_copy)

aviation_data_copy.columns


Index(['event_id', 'investigation_type', 'accident_number', 'event_date',
       'location', 'country', 'latitude', 'longitude', 'airport_code',
       'airport_name', 'injury_severity', 'aircraft_damage',
       'aircraft_category', 'registration_number', 'aircraft_make',
       'aircraft_model', 'amateur_built', 'number_of_engines', 'engine_type',
       'far_description', 'schedule_type', 'purpose_of_flight', 'air_carrier',
       'fatal_injuries', 'serious_injuries', 'minor_injuries', 'uninjured',
       'weather_condition', 'phase_of_flight', 'report_status',
       'publication_date'],
      dtype='object')

In [58]:
#make a copy of the US-subset
us_data = aviation_data_copy[aviation_data_copy['country'] == 'United States'].copy()
#make a copy for the diaspora data
diaspora_data = aviation_data_copy[aviation_data_copy['country'] != 'United States'].copy()

##  General Rules for Dropping Data

Dropping data—whether rows or columns—should be done cautiously, guided by domain knowledge and data quality goals. Below are standard, defensible rules that i will use in my analysis.

---

###  Dropping Columns

I will drop a column if:

- It has a **high percentage of missing values** (typically > 50–70%) and is not critical for analysis.
- It contains **only a single unique value** (i.e., zero variance — no information gain).
- It is a **duplicate of another column** (redundancy).
- The data is **irrelevant to the current analysis objectives** (e.g., IDs or metadata not used for joins or context).
- It is **impossible to interpret or decode** (e.g., poorly documented, encoded variables with no lookup).

---

###  Dropping Rows

I will  drop a row if:

- **Critical columns are missing**, especially where imputation is not appropriate (e.g., timestamps, unique identifiers, target variable).
- It contains **clearly erroneous or corrupted data** (e.g., wrong data types, impossible values like negative injuries or invalid dates).
- It is a **complete duplicate** of another row.
- It **violates integrity constraints**, such as conflicting values across dependent fields.

---

### Cautions

- Consider **imputation or transformation** before dropping — dropping should be the **last resort** if data is unrecoverable.
- **Document your rationale** for each drop, especially in sensitive or audit-heavy domains like aviation or healthcare.
- Consider the **impact on representativeness**: Dropping too many rows can introduce bias or reduce statistical power.

---

### Best Practice

I will use `.info()`, `.isnull().sum()`, and `.nunique()` early in EDA to assess the quality of each column and  back decisions with simple visuals (e.g., **missingness heatmaps** or **histograms**).


In [59]:

us_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82248 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   event_id             82248 non-null  object 
 1   investigation_type   82248 non-null  object 
 2   accident_number      82248 non-null  object 
 3   event_date           82248 non-null  object 
 4   location             82237 non-null  object 
 5   country              82248 non-null  object 
 6   latitude             32265 non-null  object 
 7   longitude            32255 non-null  object 
 8   airport_code         49072 non-null  object 
 9   airport_name         51570 non-null  object 
 10  injury_severity      82140 non-null  object 
 11  aircraft_damage      80269 non-null  object 
 12  aircraft_category    28154 non-null  object 
 13  registration_number  82132 non-null  object 
 14  aircraft_make        82227 non-null  object 
 15  aircraft_model       82210 non-null  obje

In [60]:
us_data.isna().any()

event_id               False
investigation_type     False
accident_number        False
event_date             False
location                True
country                False
latitude                True
longitude               True
airport_code            True
airport_name            True
injury_severity         True
aircraft_damage         True
aircraft_category       True
registration_number     True
aircraft_make           True
aircraft_model          True
amateur_built           True
number_of_engines       True
engine_type             True
far_description         True
schedule_type           True
purpose_of_flight       True
air_carrier             True
fatal_injuries          True
serious_injuries        True
minor_injuries          True
uninjured               True
weather_condition       True
phase_of_flight         True
report_status           True
publication_date        True
dtype: bool

## Findings ?
The only columns with no missing data are event_id,investigation_type,accident_number, event_date
i will go through each column one by one and keep relevant data and drop the rest, i can also use domain knowlege to fill some columns, either way i want to maintain minimal bias according to what i have.

## COLUMN BY COLUMN INVESTIGATION AND VERDICT


I noticed a pattern which might help me in one way or another fill missing data in my aviation data set, this is what i found

### The NTSB `accident_number` has a patter as shown and explained below:

According to the **NTSB Aviation Data Dictionary**, the structure of an `accident_number` follows a specific pattern:

###  Format Breakdown:

- **First 3 characters**: NTSB **office code**  
  *Example*: `MIA` = Miami Regional Office

- **Next 2 digits**: **Fiscal year** of the investigation  
  *Example*: `85` = Fiscal Year 1985

- **Next 2 letters**: **Investigation category and mode**  
  *Indicates whether the investigation involved airline, marine, etc.*

- **Next 3 digits**: A **sequential number** showing the order the case was opened in that fiscal year

- **Optional final letter**: Indicates **multiple aircraft** involved in the same event

---

### Example: `MIA85LAMS1`

This breaks down as:

- `MIA` → Miami NTSB Office  
- `85` → Fiscal year 1985  
- `L` → Likely a **major investigation** in **aviation** mode  
- `AMS` → Additional **category codes**  
- `1` → First in the sequence (possibly one of multiple aircraft)

##  Final Verdict on Missing `Location` Values (U.S. Data)

While analyzing the `Accident_Number` syntax, i came up with the following insights:

- The prefix (e.g., `MIA`, `FTW`, `LAX`) typically refers to the **NTSB regional office** that conducted the investigation — **not necessarily the accident location**.
- In some cases, the prefix aligns with the actual location.
- However, in other instances, the office may be **geographically distant** from where the accident occurred, making it **unreliable as a proxy** for true location.

---

### Conclusion

Although the `Accident_Number` can offer **hints**, it **cannot be consistently used** to infer accurate location data.

---

###  Data Cleaning Decision

To address the missing `Location` values:

- All missing values in the `location` column will be **filled with `"Unknown"`** to preserve data integrity.
- These rows will be **retained** in the dataset for now.
- Rows may only be **dropped** if deemed necessary in future analysis steps.


In [86]:
#location
us_data['location'] = us_data['location'].fillna("Unknown")
us_data.isna().sum()


event_id                   0
investigation_type         0
accident_number            0
event_date                 0
location                   0
country                    0
latitude               49983
longitude              49993
airport_code           33176
airport_name           30678
injury_severity          108
aircraft_damage         1979
aircraft_category      54094
registration_number      116
aircraft_make             21
aircraft_model            38
amateur_built             21
number_of_engines       1875
engine_type             3042
far_description        54023
schedule_type          71951
purpose_of_flight       2429
air_carrier            67753
fatal_injuries         10654
serious_injuries       11375
minor_injuries         10729
uninjured               5005
weather_condition        645
phase_of_flight        21090
report_status           2611
publication_date       12681
city                       0
state                      0
state_full               123
dtype: int64

At this stage i will split this column into two new columns before i proceed with the cleaning 

The `location` column combines city and state information in a single string ( `"COCOA, FL"`). To support more granular geographic analysis, i will split this column into two distinct fields:

- **`city`** – the name of the city, town, or municipality where the event occurred  
- **`state`** – the two-letter U.S. state abbreviation ( `FL`, `CA`)

-----

- **Missing or malformed entries**: If the `location` field is  missing or did not contain a comma, both `city` and `state` will be assigned `'Unknown'`.
- **Whitespace handling**: Leading and trailing whitespaces will be stripped from both city and state values for consistency.
- **State validation**: I will  U.S. state code reference provided  (`USState_Codes.csv`)  to map abbreviations to full state names.
- **New field – `state`**: This additional column improves interpretability and supports advanced analysis (e.g., aggregating by full state name).

By structuring the `location` data this way, we enable more precise regional breakdowns, simplify future joins with FAA and weather datasets, and enhance the overall analytical quality of the dataset.


In [ ]:
us_data[['city', 'state']] = (
    us_data['location']
    .fillna('Unknown, Unknown')            
    .str.split(',', n=1, expand=True)     
    .apply(lambda x: x.str.strip())        
)
us_data['city'] = us_data['city'].fillna('Unknown')
us_data['state'] = us_data['state'].fillna('Unknown')
us_data['state'] = us_data['state'].str.upper()

us_data.head()

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,...,fatal_injuries,serious_injuries,minor_injuries,uninjured,weather_condition,phase_of_flight,report_status,publication_date,city,state
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN,MOOSE CREEK,ID
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996,BRIDGEPORT,CA
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007,Saltville,VA
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000,EUREKA,CA
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980,Canton,OH


In [63]:
#loading the USState_Codes.csv
state_codes = pd.read_csv('Data/Aviation-data/USState_Codes.csv')
state_codes.info()
state_codes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   US_State      62 non-null     object
 1   Abbreviation  62 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


,US_State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [64]:
state_codes.rename(columns={
    'Abbreviation':'state',
    'US_State':'state_full'
}, inplace=True)
us_data = us_data.merge(state_codes, on='state', how='left')


In [85]:
us_data['city'] = us_data['city'].str.strip().str.upper()
us_data['state_full']=us_data['state_full'].str.strip().str.upper()
us_data['state']=us_data['state'].str.strip().str.upper()
us_data.head()

,event_id,investigation_type,accident_number,event_date,location,country,latitude,longitude,airport_code,airport_name,...,serious_injuries,minor_injuries,uninjured,weather_condition,phase_of_flight,report_status,publication_date,city,state,state_full
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN,MOOSE CREEK,ID,IDAHO
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996,BRIDGEPORT,CA,CALIFORNIA
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007,SALTVILLE,VA,VIRGINIA
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000,EUREKA,CA,CALIFORNIA
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980,CANTON,OH,OHIO


0     MOOSE CREEK
1      BRIDGEPORT
2       SALTVILLE
3          EUREKA
4          CANTON
5          BOSTON
6          COTTON
7         PULLMAN
8    EAST HANOVER
9    JACKSONVILLE
Name: city, dtype: object